In [1]:
import import_ipynb
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np
from bert.tokenization import FullTokenizer
from tqdm import tqdm_notebook
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
import bertkeras as bk
import BERTDictionary as bd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import pyplot
from unidecode import unidecode
from scipy.stats import norm, boxcox
pd.options.display.max_colwidth = 1000
sns.set()

importing Jupyter notebook from bertkeras.ipynb
importing Jupyter notebook from BERTDictionary.ipynb
[NEL]
nubby
transvestite
lube
lubricant
vibrate
suction
dildo
vibrator
trembler
pulsator
lockaid
buzzer
penetrator
kegel
stimulator
masturbator
lasso
bondage
pasties
strapon
didlo
dido
dilo
dlo
gadget
bdsm
sadism
masochism
squirt
clitor
pelvic
vagina
anus
rectum
scrotum
perineum


In [2]:
df= pd.read_csv('../dataset/training.csv', index_col='ID')
labels = df['label']
drop_columns = [ '﻿ASIN','parent_asin','Product Group Description', 
                      'product_name', 'brand_code', 'brand_name']
df = df.drop(columns=drop_columns)
df = df.fillna("")

cols = df.columns.tolist()
cols.remove('label')
text = df[cols].apply(' '.join, axis=1)


In [3]:
html_re = re.compile('<[^>]*>')
garbage_re = re.compile('[^0-9a-z,\'\.]+') 
weight_re = re.compile('[\d.]+[ ]?(kg+|lbs+|g+|lb+)')
garbage_threshhold = 0.0003
# check if it actually helps, tests https://regex101.com/r/xwnV5l/1
size_re = re.compile('[\d.]+([ cminch]+)?[ x*]+[\d.]+([ cminch]+)?([ x*]+[\d.]+)?([ cminch]+)')

def html_cleanup(html):
    return re.sub(html_re, ' ', html)
def lower_case(txt):
    return txt.lower()
def split_by_garbage(txt):
    return re.sub(garbage_re, ' ', txt)
def decode_chars(txt):
    garbage_fiteted = re.sub(garbage_re, '', txt)
    decoded_txt = unidecode(txt)
    return decoded_txt if len(garbage_fiteted)/len(txt) < garbage_threshhold \
        else '[NEL] ' + decoded_txt 

def cleanup_size(txt):
    return re.sub(size_re, ' ', txt).replace('size:', '')

def cleanup_weight(txt):
    return re.sub(weight_re, ' ', txt)
    
def cleanup(text):
    res = text.apply(html_cleanup).apply(decode_chars).apply(lower_case) \
              .apply(split_by_garbage).apply(cleanup_size) \
              .apply(cleanup_weight)
    return res

In [4]:
texts = cleanup(text)

In [5]:
df = pd.DataFrame( {"texts" :texts, "labels": labels} )

In [6]:
# there is error in training dataset
df.loc[196,'labels'] =1


In [7]:
txt_train, txt_test, lbl_train, lbl_test = train_test_split(
    texts, labels, test_size=0.2, random_state=42)

In [8]:
lbl_train.tail()

ID
398     0
5823    0
5943    0
3822    1
6054    0
Name: label, dtype: int64

In [9]:
k=10000
arr = np.array_split(bd.adult_toys_vocab, 25)
for a in arr :
    print(a)
    w = ' '.join(a)
    txt_train = txt_train.append(pd.Series([w], index=[k]))
    lbl_train = lbl_train.append(pd.Series([1], index=[k]))
    k = k+1


['[NEL]' 'porn' 'nubby' 'lesbian']
['gay' 'transvestite' 'transgender' 'lube']
['lubricant' 'sex' 'intercourse' 'fuck']
['vibrate' 'suction' 'lick' 'suck']
['gag' 'dildo' 'vibrator' 'condom']
['anal' 'plug' 'snake']
['trembler' 'pulsator' 'lockaid']
['strap' 'buzzer' 'rabbit']
['egg' 'penetrator' 'dual']
['kegel' 'ben' 'wa']
['stimulator' 'masturbator' 'lasso']
['bondage' 'cuff' 'pasties']
['strapon' 'didlo' 'dido']
['dilo' 'dlo' 'device']
['gadget' 'pleasure' 'intimate']
['last' 'aroused' 'climax']
['enjoyment' 'orgasm' 'bdsm']
['sadism' 'submission' 'masochism']
['realistic' 'stimulate' 'squirt']
['dick' 'cock' 'penis']
['spot' 'butt' 'ass']
['nipple' 'clitor' 'pelvic']
['vagina' 'pussy' 'anus']
['prostate' 'rectum' 'ventral']
['scrotum' 'tongue' 'perineum']


In [10]:
tokenizer = bk.create_tokenizer_from_hub_module(vocab_file='vocab.txt')
max_seq_length = 512

using vocab file from vocab.txt


In [11]:
# Convert data to InputExample format
train_examples = bk.convert_text_to_examples(txt_train, lbl_train.values)
test_examples = bk.convert_text_to_examples(txt_test, lbl_test.values)

In [12]:

# Convert to features
(train_input_ids, train_input_masks, train_segment_ids, train_labels 
) = bk.convert_examples_to_features(tokenizer, train_examples, max_seq_length=max_seq_length)
(test_input_ids, test_input_masks, test_segment_ids, test_labels
) = bk.convert_examples_to_features(tokenizer, test_examples, max_seq_length=max_seq_length)

In [13]:
train_examples[0].__dict__

{'guid': None,
 'text_a': ' nel acqua di parma blu mediterraneo arancia di capri perfume for women 2.5 oz eau de toilette spray a free vial 2.5 oz eau de toilette spray ',
 'text_b': None,
 'label': 0}

In [14]:
def build_model(max_seq_length): 
    in_id = tf.keras.layers.Input(shape=(max_seq_length,), name="input_ids")
    in_mask = tf.keras.layers.Input(shape=(max_seq_length,), name="input_masks")
    in_segment = tf.keras.layers.Input(shape=(max_seq_length,), name="segment_ids")
    bert_inputs = [in_id, in_mask, in_segment]
    
    bert_output = bk.BertLayer(n_fine_tune_layers=3, pooling="first")(bert_inputs)
    dense = tf.keras.layers.Dense(256, activation='relu')(bert_output)
    pred = tf.keras.layers.Dense(1, activation='sigmoid')(dense)
    
    model = tf.keras.models.Model(inputs=bert_inputs, outputs=pred)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    return model

In [15]:
ACCURACY_THRESHOLD = 0.99
LOSS_THRESHOLD = 0.015
class myCallback(tf.keras.callbacks.Callback): 
    def on_batch_end(self, batch, logs): 
        if(logs.get('acc') > ACCURACY_THRESHOLD and logs.get('loss')<LOSS_THRESHOLD):   
          print("\nReached %2.2f%% accuracy, so stopping training!!" %(ACCURACY_THRESHOLD*100)) 
          self.model.stop_training = True

In [16]:
model = build_model(max_seq_length)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          (None, 512)          0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        (None, 512)          0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        (None, 512)          0                                            
__________________________________________________________________________________________________
bert_layer (BertLayer)          (None, 768)          110104890   input_ids[0][0]                  
                                                                 input_masks[0][0]                
          

In [17]:
# Initialize session
sess = tf.Session()
# Instantiate variables
bk.initialize_vars(sess)
tb = tf.compat.v1.keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=24, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='batch')
stop98 = myCallback()
model.fit(
    [train_input_ids, train_input_masks, train_segment_ids], 
    train_labels,
    validation_data=([test_input_ids, test_input_masks, test_segment_ids], test_labels),
    callbacks = [ tb, stop98], #, 
    epochs=8,
    batch_size=24
)

Train on 4413 samples, validate on 1098 samples
Epoch 1/8
4413/4413 [==============================] - 283s 64ms/sample - loss: 0.1955 - acc: 0.9356 - val_loss: 0.0886 - val_acc: 0.9763
Epoch 2/8
4413/4413 [==============================] - 280s 64ms/sample - loss: 0.0600 - acc: 0.9823 - val_loss: 0.0366 - val_acc: 0.9891
Epoch 3/8

Reached 99.00% accuracy, so stopping training!!
  24/4413 [..............................] - ETA: 3:57 - loss: 0.0026 - acc: 1.0000

In [18]:
model.save('restricted.h5')

In [19]:
df_subm= pd.read_csv('../dataset/public_test_features.csv', index_col='ID')
df_subm = df_subm.drop(columns=drop_columns)
df_subm = df_subm.fillna("")

text_subm = df_subm[cols].apply(' '.join, axis=1)


In [20]:
texts_subm = cleanup(text_subm)

In [21]:
subm_examples = bk.convert_text_to_examples(texts_subm, [0]*len(texts_subm))

In [22]:
(subm_input_ids, subm_input_masks, subm_segment_ids, subm_labels
) = bk.convert_examples_to_features(tokenizer, subm_examples, max_seq_length=max_seq_length)

In [23]:
subm_examples[0].text_a

' nel vintage mama e hija amor corazon cadena collar mujeres dia de la madre regalo especificaciones disea o de mama e hija, colgante con forma de coraza3n. hecho de material de aleacia3n de alta calidad con mano de obra fina. regalo perfecto para el daa de la madre u otras ocasiones. tipo collar ganero mujeres tema estilo vintage material aleacia3n de ocasiones fiesta, daa de la madre, regalo, vida diaria, etc. caracterasticas colgante de coraza3n, patra3n de letras, regalo para madre longitud de la cadena  19,69 pulgadas 1,97 pulgadas aprox. notas debido a la diferencia de configuracia3n de la luz y la pantalla, el color del artaculo puede ser ligeramente diferente al de las ima genes. puede haber una ligera diferencia de dimensia3n debido a las diferentes medidas manuales. el paquete incluye 1 x collar item specifics fino o de moda moda tipo de articulo collares nombre de la marca bluelans estilo trendy forma patron carta tipo de cadena cadena de eslabones tamano colgante 5cm compat

In [24]:
testCount = len(subm_input_ids)
def pickMax(arr):
  m = max(arr)
  return [x for x in arr]

threshold = 0.4
def tolabel(arr):
  # return int(max(arr)>=threshold)
  return max(arr)
    


In [25]:
predictions = model.predict([subm_input_ids[0:testCount], 
                                subm_input_masks[0:testCount], 
                                subm_segment_ids[0:testCount]])

In [26]:
pred = [ tolabel(p) for p in predictions]

In [27]:
results = pd.DataFrame({"text":texts_subm, "label":pred})

In [28]:
results[results['label']>0.03][results['label']<0.95]


/home/drmegavolt/miniconda3/envs/restricted_products/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,text,label
ID,,
3575,"nel hismith premium sex machine, standard hismith focus on manufacturing sex machine, we produce machine series and matched accessories.we choose the best long life and mute motor as the core of the machine,ensure the power of our products,30 n cm driving force,never stop when encounter resistance.our power supply can support wide voltage 110 250v ,power switch with precise dial. as your wish,the thrust depth can be changed through change the postion of the torsion bar.besides,you can change the masburation postion,by change the postion of support connector.the body of the machine are aluminium alloy material,with high density anodizing technology.every component is produced by precision high precision cnc,sla laser series machine,and mirageblack painted surfaces,bright in color and durable,surfaces look fresh as a new one after long time use.solid stainless chromium plated iron rod support strength is over .working life years with correct usage and cleansing maintenance. produc...",0.090924
3039,"nel hismith premium sex machine, fucking machine with remote control, 8 silicone dildo including hismith focus on manufacturing sex machine, we produce machine series and matched accessories.we choose the best long life and mute motor as the core of the machine,ensure the power of our products,30 n cm driving force,never stop when encounter resistance.our power supply can support wide voltage 110 250v ,power switch with precise dial. as your wish,the thrust depth can be changed through change the postion of the torsion bar.besides,you can change the masburation postion,by change the postion of support connector.the body of the machine are aluminium alloy material,with high density anodizing technology.every component is produced by precision high precision cnc,sla laser series machine,and mirageblack painted surfaces,bright in color and durable,surfaces look fresh as a new one after long time use.solid stainless chromium plated iron rod support strength is over .working life yea...",0.162470
4607,"nel dadou blue pens double ended did lo amal male female dong pens dadou functions promotes healthy ro tate function head designed for ultimate pleasure and ro tate st mul ti n made with the highest quality material available easy to clean and store super stretchy and silky smooth to the touch ergonomically designed to perfectly rest on the ro tate make all of your aial fantasies come true tips 1. please allow slight measurement deviation for the data lean before and after each use recommended to use with lubricants ompatible with all lubricant types wash with toy cleaner or warm water. do not shy away from this great aial as it can be used by both men and women. the silky smooth texture helps glide the bitt plug right in. featur ing a re listic tip, you are guaranteed to hit all the right pleasure points. aial plug has nice size and comfortable shape, made of medical silicone, safty, nontoxic and odorless. apply to both beginners and advanced users for explosive g p r gasm and ai...",0.697702
1486,"nel sexxxy wearable realistic strap on dildo silicone pu leather strapon dildo pants sexy belt for lesbian couple women flesh features 1 item type wearable realistic dildo 2 color flesh 3 length dildo diameter et weight 6 feature can be adsorbed and strop on strong suction cup please allow 0.3cm error due to manual measurement, thank you. flexible and adjustable the straps can be adjusted for various sizes. the back fits like a thong and the front has a small dildo that goes inserted into your vagina. soft silicone material the dildo is made of medical grade silicone and non toxic dildo, without odor, seamless and safe to use waterproof design it is easy to use and convenient to clean. double pleasure this double dildo strap on is great for the lesbian partner to enjoy giving as well as receiving at the same time.while using this strap on you will not only pleasure the partner but you will also pleasure yourself at the sam

In [29]:
results.to_csv('results.csv', columns=[ 'label'])

In [30]:
bd.adult_toys_vocab

['[NEL]',
 'porn',
 'nubby',
 'lesbian',
 'gay',
 'transvestite',
 'transgender',
 'lube',
 'lubricant',
 'sex',
 'intercourse',
 'fuck',
 'vibrate',
 'suction',
 'lick',
 'suck',
 'gag',
 'dildo',
 'vibrator',
 'condom',
 'anal',
 'plug',
 'snake',
 'trembler',
 'pulsator',
 'lockaid',
 'strap',
 'buzzer',
 'rabbit',
 'egg',
 'penetrator',
 'dual',
 'kegel',
 'ben',
 'wa',
 'stimulator',
 'masturbator',
 'lasso',
 'bondage',
 'cuff',
 'pasties',
 'strapon',
 'didlo',
 'dido',
 'dilo',
 'dlo',
 'device',
 'gadget',
 'pleasure',
 'intimate',
 'last',
 'aroused',
 'climax',
 'enjoyment',
 'orgasm',
 'bdsm',
 'sadism',
 'submission',
 'masochism',
 'realistic',
 'stimulate',
 'squirt',
 'dick',
 'cock',
 'penis',
 'spot',
 'butt',
 'ass',
 'nipple',
 'clitor',
 'pelvic',
 'vagina',
 'pussy',
 'anus',
 'prostate',
 'rectum',
 'ventral',
 'scrotum',
 'tongue',
 'perineum']